In [1]:
using KitBase, Plots, JLD2, Distributions, LinearAlgebra, Flux
using Flux: onecold, @epochs

┌ Info: Kinetic will run serially
└ @ KitBase /home2/vavrines/.julia/packages/KitBase/5jIce/src/KitBase.jl:61
┌ Info: Kinetic will run with CUDA
└ @ KitBase /home2/vavrines/.julia/packages/KitBase/5jIce/src/KitBase.jl:65
┌ Info: 1: NVIDIA GeForce RTX 3070
└ @ KitBase /home2/vavrines/.julia/packages/KitBase/5jIce/src/KitBase.jl:67
┌ Info: Scalar operation is disabled in CUDA
└ @ KitBase /home2/vavrines/.julia/packages/KitBase/5jIce/src/KitBase.jl:69


In [2]:
function regime_data(ks, w, prim, swx, swy, f)
    Mu, Mv, Mxi, _, _1 = gauss_moments(prim, ks.gas.K)
    a = pdf_slope(prim, swx, ks.gas.K)
    b = pdf_slope(prim, swy, ks.gas.K)
    swt = -prim[1] .* (moments_conserve_slope(a, Mu, Mv, Mxi, 1, 0) .+ moments_conserve_slope(b, Mu, Mv, Mxi, 0, 1))
    A = pdf_slope(prim, swt, ks.gas.K)
    tau = vhs_collision_time(prim, ks.gas.μᵣ, ks.gas.ω)
    
    fr = chapman_enskog(ks.vs.u, ks.vs.v, prim, a, b, A, tau)
    L = norm((f .- fr) ./ prim[1])

    sw = (swx.^2 + swy.^2).^0.5
    x = [w; sw; tau]
    y = ifelse(L <= 0.005, [1.0, 0.0], [0.0, 1.0])

    return x, y
end

function regime_number(Y, rg=1)
   idx = 0
    for i in axes(Y, 2)
       if Y[1, i] == rg
            idx += 1
        end
    end
    println("NS regime: $(idx) of $(size(Y, 2))")
    return nothing
end

function accuracy(nn, X, Y)
    Y1 = nn(X)

    YA1 = [onecold(Y1[:, i]) for i in axes(Y1, 2)]
    YA = [onecold(Y[:, i]) for i in axes(Y, 2)]

    accuracy = 0.0
    for i in eachindex(YA)
        if YA[i] == YA1[i]
            accuracy += 1.0
        end
    end
    accuracy /= length(YA)

    return accuracy
end

accuracy (generic function with 1 method)

In [7]:
file = open("/home2/vavrines/Coding/KiT-RT/result/logs/test_a10_ev5.csv")
data = []
for line in eachline(file)
    a = split(line, ",")
    b = [parse(Float64, a[i]) for i = 2:length(a)]
    push!(data, b)
end

In [14]:
data[1] |> minimum

-1.999876940123

In [9]:
data[2]

40000-element Vector{Float64}:
  0.000188396899
  0.000565144214
  0.000941752088
  0.001318127598
  0.00169417788
  0.00206981015
  0.002444931724
  0.002819450048
  0.003193272715
  0.00356630749
  0.003938462332
  0.004309645417
  0.004679765162
  ⋮
 -0.359310470138
 -0.328363615832
 -0.297335742717
 -0.266234506457
 -0.235067580816
 -0.203842655767
 -0.172567435592
 -0.141249636986
 -0.109896987144
 -0.078517221866
 -0.047118083638
 -0.015707319728

In [13]:
data[4] |> minimum

2.3196276e-5